In [ ]:
# test - tokenizer

import re
#entries = re.split("\n+", open( "LER/sler_1801_80_1_cw80_sx1.sad").read())
text = open( "LER/sler_1801_80_1_cw80_sx1.sad").read()
#cmd = entries[11]
#cmd =    "QKELP   =(L =.3444     K1 =1.5173117905370072e-05    ROTATE =-45 DEG     F1 =.211  FRINGE =3 )"

from sad import tokenize
for token in tokenize(text):
    print(token)

In [1]:
# tests - parsing single lines

from sad import tokenize, process_stack, LatticeObject
from numpy import pi



# parse this test string; then create a test; then go to next example
#cmd =    "QKELP   =(L =.3444     K1 =1.5173117905370072e-05    ROTATE =-45 DEG     F1 =.211  FRINGE =3 );"
#cmd =  " LZVQF2P30=(L =.3444 )       LZHQT1FOP1=(L =.3444 )      LT0SP   =(L =1.8228861071304 )     LZVQT2FOP1=(L =.3444 );"
#cmd = "  DRIFT  LBC1LP  =(L =.031647501274725515 )"
cmd = "  DRIFT  LBC1LP  =(L =.031647501274725515 ) LXL28467=(L =.28467 )       LBLC1LP1A=(L =.11967003059421588 )"
token_stack = []

for token in tokenize(cmd): # reverse the stack for parsing and create a list
    token_stack = [token] + token_stack

print(token_stack)

objs, line_def = process_stack(token_stack)
for o in objs:
    print(f"{o.type} : {o}")
print(f"line : {line_def}")


[Token(type='RBR', value=')', line=1, column=105), Token(type='NUMBER', value=0.11967003059421588, line=1, column=86), Token(type='EQUAL', value='=', line=1, column=85), Token(type='ID', value='L', line=1, column=83), Token(type='LBR', value='(', line=1, column=82), Token(type='EQUAL', value='=', line=1, column=81), Token(type='ID', value='LBLC1LP1A', line=1, column=72), Token(type='RBR', value=')', line=1, column=64), Token(type='NUMBER', value=0.28467, line=1, column=57), Token(type='EQUAL', value='=', line=1, column=56), Token(type='ID', value='L', line=1, column=54), Token(type='LBR', value='(', line=1, column=53), Token(type='EQUAL', value='=', line=1, column=52), Token(type='ID', value='LXL28467', line=1, column=44), Token(type='RBR', value=')', line=1, column=42), Token(type='NUMBER', value=0.031647501274725515, line=1, column=22), Token(type='EQUAL', value='=', line=1, column=21), Token(type='ID', value='L', line=1, column=19), Token(type='LBR', value='(', line=1, column=18), T

In [1]:
### testing - full file

from sad import *

#input_file = "test_sad_2.sad"
#input_file = "LER/sler_1801_80_1_cw80_sx1.sad"
input_file = "TL/linac-btp-truncated.sad"
print(f"Reading the input file: {input_file}")

sad_object = read_sad(input_file)
print(f"read object {sad_object}")


Reading the input file: TL/linac-btp-truncated.sad
statement read, processing stack...
ERROR, printing stack
[Token(type='END', value=';', line=1, column=1)]
line : []
statement read, processing stack...
DRIFT : DRIFT : LKKE  :  {'L': 0.2}
DRIFT : DRIFT : LKE1U  :  {'L': 0.26387109923590146}
DRIFT : DRIFT : LQRD1U  :  {'L': 0.0018699999999999828}
DRIFT : DRIFT : LQRD1D  :  {'L': 0.08187}
DRIFT : DRIFT : LR1U1  :  {'L': 0.07049999999999997}
DRIFT : DRIFT : LR1U2  :  {'L': 0.2645}
DRIFT : DRIFT : LR1D  :  {'L': 1.077}
DRIFT : DRIFT : LRMID  :  {'L': 2.2}
DRIFT : DRIFT : LR2UU  :  {'L': 0.855}
DRIFT : DRIFT : LR2DU1  :  {'L': 0.222}
DRIFT : DRIFT : LR2DU2  :  {'L': 0.2929999999999935}
DRIFT : DRIFT : LR2DD  :  {'L': 0.03500000000000003}
DRIFT : DRIFT : LHRUU  :  {'L': 0.0034732000000000044}
DRIFT : DRIFT : LHRUD  :  {'L': 0.0015268}
DRIFT : DRIFT : LQRFU  :  {'L': 0.06984499999999999}
DRIFT : DRIFT : LQS  :  {'L': 0.224845}
DRIFT : DRIFT : LSE2B  :  {'L': 2.5145}
DRIFT : DRIFT : LSE2A  : 

In [2]:
print(f"{sad_object.lattice_objects[2]}")

DRIFT : LQRD1U  :  {'L': 0.0018699999999999828}


In [4]:
# building lattice objects

f=open("test_optics.py","w")

f.write("from ocelot.cpbd.elements import *\n#elements\n")


for o in sad_object.lattice_objects:

    if o.type == "DRIFT":
        f.write(f"{o.name} = Drift(eid=\"{o.name}\",l={o.get_parameter('L')})\n")
        continue
    if o.type == "MONI":
        f.write(f"{o.name} = Monitor(eid=\"{o.name}\",l={o.get_parameter('L')})\n")
        continue
    if o.type == "MARK":
        f.write(f"{o.name} = Marker(eid=\"{o.name}\")\n")
        continue
    if o.type == "BEND":
        e1 = o.get_parameter('E1') * o.get_parameter('ANGLE')
        e2 = o.get_parameter('E2') * o.get_parameter('ANGLE')
        if 'ROTATE' in o.parameters.keys():
            f.write(f"{o.name} = SBend(eid=\"{o.name}\",l={o.get_parameter('L')},angle={o.get_parameter('ANGLE')},e1={e1},e2={e2},tilt={o.get_parameter('ROTATE')})\n")
        else:
            f.write(f"{o.name} = SBend(eid=\"{o.name}\",l={o.get_parameter('L')},angle={o.get_parameter('ANGLE')},e1={e1},e2={e2})\n")
        continue
    if o.type == "QUAD":
        f.write(f"{o.name} = Quadrupole(eid=\"{o.name}\",l={o.get_parameter('L')},k1={o.get_parameter('K1')/o.get_parameter('L')},tilt={o.get_parameter('ROTATE')})\n")
        continue
    if o.type == "SEXT":
        f.write(f"{o.name} = Sextupole(eid=\"{o.name}\",l={o.get_parameter('L')},k2={o.get_parameter('K2')},tilt={o.get_parameter('ROTATE')})\n")
        continue

    if o.type == "MULT": # So far replacing with quadrupole, todo: check out Rf and quad overlay
        if o.get_parameter('L') > 0:
            f.write(f"{o.name} = Quadrupole(eid=\"{o.name}\",l={o.get_parameter('L')},k1={o.get_parameter('K1')/o.get_parameter('L')},tilt={o.get_parameter('ROTATE')})\n")
        else: 
            f.write(f"{o.name} = Drift(eid=\"{o.name}\",l={o.get_parameter('L')})\n")
        continue

    if o.type == "SOL": # So far ignoring parameters
        f.write(f"{o.name} = Solenoid(eid=\"{o.name}\",l={o.get_parameter('L')})\n")
        continue
    if o.type == "CAVI": # phase factor for positrons!
        f.write(f"{o.name} = Cavity(eid=\"{o.name}\",l={o.get_parameter('L')},freq={o.get_parameter('FREQ')}, v={ o.get_parameter('VOLT')*1.e-9}, phi={90+o.get_parameter('PHI')*180/pi})\n")
        continue




    f.write(f"{o}\n")

f.write("END = Marker()\n")

f.write("#lattice def\nlattice_list = (")


for e in sad_object.lattice_list:
    f.write(f"{e}, ")

f.write("END)")


f.close()

